In [ ]:
import random
import time
import base64
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request

# Email address and label name
GMAIL_ID = "edin.bro89@gmail.com"
LABEL_NAME = "Auto Responded"

# Message
EMAIL_REPLY = "Thank you for your email. I am currently on vacation and will respond to your message when I return."

# Time intervals (in seconds)
MIN_INTERVAL = 45
MAX_INTERVAL = 120

# Scope required for accessing Gmail
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]


def authenticate_gmail():
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    return creds


def check_emails(service):
    try:
        results = service.users().messages().list(userId="me", labelIds=["INBOX"]).execute()
        messages = results.get("messages", [])
        for message in messages:
            email = service.users().messages().get(userId="me", id=message["id"]).execute()
            thread_id = email["threadId"]
            if not email.get("replies"):
                send_email_reply(service, thread_id)
                add_label_to_email(service, message["id"], LABEL_NAME)
    except Exception as e:
        print("An error occurred while checking emails:", str(e))



def send_email_reply(service, thread_id):
    message = {
        "userId": "me",
        "threadId": thread_id,
        "raw": base64.urlsafe_b64encode(
            f"From: {GMAIL_ID}\nTo: {GMAIL_ID}\nSubject: Re: Auto Response\n\n{EMAIL_REPLY}".encode("utf-8")
        ).decode("utf-8")
    }
    service.users().messages().send(userId="me", body=message).execute()




def add_label_to_email(service, email_id, label_name):
    labels = service.users().labels().list(userId="me").execute()["labels"]
    label_id = None
    for label in labels:
        if label["name"] == label_name:
            label_id = label["id"]
            break
    if not label_id:
        label = service.users().labels().create(
            userId="me", body={"name": label_name}
        ).execute()
        label_id = label["id"]
    service.users().messages().modify(
        userId="me", id=email_id, body={"addLabelIds": [label_id]}
    ).execute()


def main():
    creds = authenticate_gmail()
    service = build("gmail", "v1", credentials=creds)
    while True:
        check_emails(service)
        interval = random.randint(MIN_INTERVAL, MAX_INTERVAL)
        time.sleep(interval)


if __name__ == "__main__":
    main()
